# Identify gaps in CARE service relative to unsheltered population
* Map number of CARE/CARE+ requests by tract (2017-2019)
* Map number of homeless counts by tract (2017-2019)
* Map difference: (total CARE/CARE+ for homeless encampments) - (total homelessness)

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl, basemaps
import json
import requests
from ipywidgets import link, FloatSlider
from branca.colormap import linear
import boto3
import folium

In [2]:
catalog = intake.open_catalog('../catalogs/*.yml')
bucket_name = 's3://city-of-los-angeles-data-lake/public-health-dashboard/'
s3 = boto3.client('s3')

## Import files

In [3]:
homeless = gpd.read_file(f'{bucket_name}gis/raw/homelessness_lacity_2017_2019.geojson').to_crs({'init':'epsg:4326'})
# Only keep the tracts that are withn City of LA
homeless = homeless[homeless.CD != 0]

In [4]:
pop = pd.read_parquet(f'{bucket_name}data/raw/pop_by_tract2017.parquet') 

In [5]:
care = gpd.read_file(f'{bucket_name}gis/intermediate/care311_tracts.geojson').to_crs({'init':'epsg:4326'})

## Merge dfs

In [6]:
# Merge homelessness and population
m1 = pd.merge(homeless, pop, on = 'GEOID', how = 'inner', validate = 'm:1')

In [7]:
# Merge in CARE 311 service requests
m2 = pd.merge(m1, care, on = ['GEOID', 'year'], how = 'left', validate = '1:1')

In [8]:
# Keep the geometry from homelessness because it's been clipped to City of LA
m2 = m2.drop(columns = ['geometry_y'])

m2.rename(columns = {'geometry_x': 'geometry', 'GEOID': 'id'}, inplace = True)

In [9]:
# Fill in NaNs with zeroes for CARE service requests
for col in ['bulky', 'homeless', 'illegal', 'other']:
    m2[col] = m2[col].fillna(0)    

In [10]:
pivot1 = m2.dissolve(by = ['id', 'clipped_area', 'full_area', 
                           'CD', 'pop', 'SPA', 'SD'], aggfunc = 'sum').reset_index().drop(columns = ['year'])

pivot1.head()

,id,clipped_area,full_area,CD,pop,SPA,SD,geometry,totUnshelt,totShelt,totPeople,bulky,homeless,illegal,other
0,06037101110,0.441019,0.441019,7,4566.0,2,5,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",29.498,0,29.498,1.0,3.0,0.0,0.0
1,06037101122,1.020722,1.020722,7,3064.0,2,5,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4.618,0,4.618,0.0,0.0,0.0,0.0
2,06037101210,0.251197,0.251197,7,6043.0,2,5,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",41.897,0,41.897,1.0,3.0,6.0,1.0
3,06037101220,0.269802,0.269802,7,3340.0,2,5,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",72.464,0,72.464,2.0,1.0,1.0,0.0
4,06037101300,0.995075,0.996474,7,4285.0,2,5,"POLYGON ((-118.27822 34.25068, -118.27822 34.2...",36.710,0,36.710,2.0,8.0,5.0,0.0


In [11]:
pivot1.to_file(driver = 'GeoJSON', filename = '../gis/homelessness_tract.geojson')
s3.upload_file('../gis/homelessness_tract.geojson', 'city-of-los-angeles-data-lake', 
               'public-health-dashboard/gis/intermediate/homelessness_tract.geojson')

In [20]:
len(pivot1)

994

## Map homelessness

In [12]:
tracts = pivot1[['id', 'geometry']]
tracts['GEOID'] = tracts['id']
tracts = tracts.set_index('id')
tracts.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,geometry,GEOID
id,,
06037101110,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",06037101110
06037101122,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",06037101122
06037101210,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",06037101210
06037101220,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",06037101220
06037101300,"POLYGON ((-118.27822 34.25068, -118.27822 34.2...",06037101300


In [13]:
# to_json converts the gdf to json. json.loads converts it into dictionary
geo_data = json.loads(tracts.to_json())

# Take what we want to map and turn it into a dictionary
unsheltered = dict(zip(pivot1['id'].tolist(), pivot1['totUnshelt'].tolist()))

layer = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = unsheltered,
    colormap = linear.viridis,
    border_color = 'white',
    style = {'fillOpacity': 0.8}
)


m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 10,
                  basemap = basemaps.CartoDB.Positron)
m.add_layer(layer)

m

Map(basemap={'url': 'http://c.basemaps.cartocdn.com/light_all/{z}/{x}/{y}.png', 'max_zoom': 20, 'attribution':…

In [14]:
pivot1.head()

,id,clipped_area,full_area,CD,pop,SPA,SD,geometry,totUnshelt,totShelt,totPeople,bulky,homeless,illegal,other
0,06037101110,0.441019,0.441019,7,4566.0,2,5,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",29.498,0,29.498,1.0,3.0,0.0,0.0
1,06037101122,1.020722,1.020722,7,3064.0,2,5,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4.618,0,4.618,0.0,0.0,0.0,0.0
2,06037101210,0.251197,0.251197,7,6043.0,2,5,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",41.897,0,41.897,1.0,3.0,6.0,1.0
3,06037101220,0.269802,0.269802,7,3340.0,2,5,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",72.464,0,72.464,2.0,1.0,1.0,0.0
4,06037101300,0.995075,0.996474,7,4285.0,2,5,"POLYGON ((-118.27822 34.25068, -118.27822 34.2...",36.710,0,36.710,2.0,8.0,5.0,0.0


In [15]:
unshelt_df = pivot1[['id', 'totUnshelt']]
unshelt_df.rename(columns = {'id': 'GEOID'}, inplace = True)
#geo = pivot1[['id', 'geometry']]
 
m = folium.Map(
    location=[34.0536, -118.2427], 
    zoom_start=12, 
    tiles='cartodbpositron'
)

folium.Choropleth(
    geo_data=pivot1,
    name = 'choropleth',
    data=unshelt_df,
    columns=['GEOID', 'totUnshelt'],
    key_on='feature.properties.id',
    fill_color='BuPu',
    fill_opacity=0.95,
    line_opacity=0.8, line_color='white',
    bins=[0, 50, 100, 500, 4000],
    legend_name='Housing Units per Acre'
).add_to(m)


#m.add_child(folium.LayerControl())

m.save('map.html')

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [16]:
pivot1['ratio'] = pivot1.apply(lambda row: row.totUnshelt / row.homeless if row.homeless > 0 else np.nan, axis = 1)

In [17]:
pivot1.ratio.describe()

count    909.000000
mean       3.784405
std        8.378075
min        0.000000
25%        0.669117
50%        1.500154
75%        3.406923
max      106.573000
Name: ratio, dtype: float64

In [18]:
pivot1

,id,clipped_area,full_area,CD,pop,SPA,SD,geometry,totUnshelt,totShelt,totPeople,bulky,homeless,illegal,other,ratio
0,06037101110,0.441019,0.441019,7,4566.0,2,5,"POLYGON ((-118.30229 34.25870, -118.30091 34.2...",29.498,0,29.498,1.0,3.0,0.0,0.0,9.832667
1,06037101122,1.020722,1.020722,7,3064.0,2,5,"POLYGON ((-118.30334 34.27371, -118.30330 34.2...",4.618,0,4.618,0.0,0.0,0.0,0.0,NaN
2,06037101210,0.251197,0.251197,7,6043.0,2,5,"POLYGON ((-118.29945 34.25598, -118.29792 34.2...",41.897,0,41.897,1.0,3.0,6.0,1.0,13.965667
3,06037101220,0.269802,0.269802,7,3340.0,2,5,"POLYGON ((-118.28592 34.25227, -118.28592 34.2...",72.464,0,72.464,2.0,1.0,1.0,0.0,72.464000
4,06037101300,0.995075,0.996474,7,4285.0,2,5,"POLYGON ((-118.27822 34.25068, -118.27822 34.2...",36.710,0,36.710,2.0,8.0,5.0,0.0,4.588750
5,06037101400,2.435987,2.435987,7,4089.0,2,5,"POLYGON ((-118.32238 34.24963, -118.32212 34.2...",33.610,0,33.610,2.0,15.0,4.0,0.0,2.240667
6,06037102103,0.458043,0.458043,2,1838.0,2,5,"POLYGON ((-118.36533 34.22870, -118.36396 34.2...",9.924,0,9.924,1.0,0.0,2.0,0.0,NaN
7,06037102104,0.639617,0.640284,2,3681.0,2,5,"POLYGON ((-118.35620 34.21971, -118.35594 34.2...",1.425,0,1.425,1.0,6.0,1.0,0.0,0.237500
8,06037102105,0.190095,0.190232,2,1956.0,2,5,"POLYGON ((-118.35307 34.20878, -118.35307 34.2...",41.021,0,41.021,0.0,7.0,4.0,0.0,5.860143
9,06037102107,4.988642,4.988642,7,3862.0,2,5,"POLYGON ((-118.36789 34.23939, -118.36788 34.2...",30.796,0,30.796,2.0,24.0,3.0,0.0,1.283167
